## Fx Forward Upsert and Valuation example notebook

This notebook displays the upsert and valuation functionality in Lusid in the context of FX forwards. We will be upserting the forwards as transactions and as holdings. Finally, we will define a LUSID recipe and run a valuation

## Table of contents

- 1. [Setup](#1.Setup)
   * [1.1 Create Portfolio](#1.1-Create-portfolio)
   * [1.2 Fund portfolio with cash](#1.2-Fund-Portfolio-with-cash)
- 2. [2.Upsert as transaction](#2.Upsert-FX-Forward-position-as-Transaction)
- 3. [3.Upsert as holding](##3.-Load-Fx-Forward-as-Holding)
- 4. [Valuation](##-4.-Valuation)
   * [4.1 Upsert Fwd quote](###-4.1-Quote-FWD-Rate)
   * [4.2 Upsert Spot quote](###-4.2-Quote-for-spot)
   * [4.3 Define Recipe for valuation](##-4.3-Define-Recipe-for-valuation)
   * [4.4 Run valuation](##-4.4.-Run-Valuation)

In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Fx Forward upsert and valuation in Lusid

Attributes
----------
Transaction Portfolios
Quotes
Holdings
Transactions
Valuation
"""

toggle_code("Toggle Docstring")


In [2]:
import os
import lusid
import lusid.models as lm
from lusid.utilities import ApiClientFactory
from lusid import api
from lusidjam import RefreshingToken
import pandas as pd
from datetime import date
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
import json
from lusid import ApiException

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook")

In [5]:
transactions_api = api_factory.build(lusid.TransactionPortfoliosApi)
quotes_api = api_factory.build(lusid.QuotesApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)
portfolios_api = api_factory.build(lusid.PortfoliosApi)

In [36]:
scope ='FX_FW_Transact'
portfolio_code= 'Fx_Port'

# 1.Setup

## 1.1 Create portfolio

Create a portfolio in Lusid to hold the Fx Forwards. To do this, we use the <a id='https://www.lusid.com/docs/api#tag/Transaction-Portfolios'>Transaction Portfolios API</a>.

In [39]:
try:
    transactions_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request = lm.CreateTransactionPortfolioRequest(
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency='GBP',
                created=str(date(2020,1,1))))
        
except ApiException as e:
    display(json.loads(e.body)["title"])

"Could not create a portfolio with id 'Fx_Port' because it already exists in scope 'FX_FW_Transact'."

## 1.2 Fund Portfolio with cash

Fund portfolio with initial an cash balance. This is done by upserting a 'FundsIn' transaction of 10000 units of GBP

In [43]:
response = transactions_api.upsert_transactions('FX_FW_Transact',
                                     Port1_Code,
                                     transaction_request=[lm.TransactionRequest(
                                        transaction_id='Intial_funding',
                                        type='FundsIn',
                                        instrument_identifiers= {"Instrument/default/Currency":'GBP'},
                                        transaction_date= str(date(2020,1,1)),
                                        settlement_date= str(date(2020,1,1)),
                                        units=10000,
                                        total_consideration= lm.CurrencyAndAmount(10000,'GBP'))])

## 2. Load Fx Forward as Holding

Load FX Forward as holdings- each leg of the forward is loaded in as a separate holding (within the taxlot data structure_).  They are joined by the "Holding/default/ForwardTransactionId" property. This is done using  <a id='https://www.lusid.com/docs/api#operation/AdjustHoldings'>Adjust Holdings</a> . you can learn more about the Transaction Portfolios data model <a id='https://www.lusid.com/docs/api#section/Data-Model/Transaction-Portfolios'>here</a>.

In [12]:
class FxForwardHoldingLoader():
    
    def __init__(self, filename,api_factory, scope, portfolio, property_scope):
        self.filename = filename
        self.scope = scope
        self.portfolio = portfolio
        self.property_scope = property_scope
        self.transaction_portfolios_api = api_factory.build(api.TransactionPortfoliosApi)

        fwd_leg_df= pd.read_csv(self.filename)
    
    '''Submit adjust fx forward holdings requests'''
    def adjust_fx_forward_holdings(self, effective_at, adjust_holdings_requests):
        display(lusid_response_to_data_frame(adjust_holdings_requests))
        response = self.transaction_portfolios_api.adjust_holdings(
                scope=self.scope,
                code=self.portfolio,
                effective_at = effective_at,
                adjust_holding_request=adjust_holdings_requests,
                reconciliation_methods=["FxForward"]
            )
        response= response.to_dict()
        display({'response link':response['links'][2]['href']})


    ''' Create fx forward adjust holding requests from a single entry for a Fwd'''
    def _create_adjust_holdings_requests(self, fx_fwd):
        adjust_holdings_requests = [
            # create dom ccy fwd request
            self._create_adj_holding_request(fx_fwd, True),
            # create dom ccy fgn request
            self._create_adj_holding_request(fx_fwd, False)
        ]
        return adjust_holdings_requests;

    def _create_adj_holding_request(self, fwd_leg_df, is_dom=True):
        ccy = fwd_leg_df["Ccy"] if is_dom else fwd_leg_df["Ccy2"]
        purchase_date = pd.to_datetime(fwd_leg_df['TradeDate'], utc=True, format=DATE_FORMAT).isoformat()
        settlement_date = pd.to_datetime('1/5/2021', utc=True, format=DATE_FORMAT).isoformat()
        instrument_identifiers = {
            'Instrument/default/Currency': ccy
        }
        properties = {
            # crucial fxforward property to link both legs of fx forward
            "Holding/default/ForwardTransactionId":
                self._create_perp_property("Holding/default/ForwardTransactionId", fwd_leg_df["TradeNumber"])
        }
        tax_lots = [

            # leg
            lm.TargetTaxLotRequest(
                ####The number of units of the instrument in this tax-lot.
                units=(fwd_leg_df["CcyAmount"]*-1) if not is_dom else fwd_leg_df["Ccy2Amount"],
                #### Cost- An amount of a specific currency, specifying a value and an associated unit
                cost=lm.CurrencyAndAmount((fwd_leg_df["CcyAmount"]*-1), fwd_leg_df["Ccy"]) if not is_dom else lm.CurrencyAndAmount(
                    fwd_leg_df["Ccy2Amount"], fwd_leg_df["Ccy2"]),
                ###The total cost of the tax-lot in the transaction portfolio's base currency.
                portfolio_cost=fwd_leg_df["CostFC"],
                #### The purchase price of each unit of the instrument held in this tax-lot. This forms part of the unique key required for multiple tax-lots.
                price=fwd_leg_df["Price"],
                #### The purchase date of this tax-lot. This forms part of the unique key required for multiple tax-lots.
                purchase_date=purchase_date,
                #####The settlement date of the tax-lot's opening transaction.
                settlement_date=settlement_date,
            )
        ]
        sub_holding_keys=None
        return lm.AdjustHoldingRequest(
            instrument_identifiers,
            sub_holding_keys,
            properties,
            tax_lots
        )

    def _create_perp_property(self, key, value):
        return  lm.PerpetualProperty(key, lm.PropertyValue(label_value=value))

In [13]:
fx_fwds = pd.read_csv('data/Fwd_data/fx_fwd_sample.csv')


for index, fx in corporate_action_df.iterrows():
    # First leg
    ccy = fx["Ccy"]
    purchase_date = pd.to_datetime(fwd_leg_df['TradeDate'], utc=True, format=DATE_FORMAT).isoformat()
    settlement_date = pd.to_datetime('1/5/2021', utc=True, format=DATE_FORMAT).isoformat()
    instrument_identifiers = {
            'Instrument/default/Currency': ccy
        }
        properties = {
            # crucial fxforward property to link both legs of fx forward
            "Holding/default/ForwardTransactionId":
                self._create_perp_property("Holding/default/ForwardTransactionId", fwd_leg_df["TradeNumber"])
        }
        tax_lots = [

            # leg
            lm.TargetTaxLotRequest(
                ####The number of units of the instrument in this tax-lot.
                units=(fwd_leg_df["CcyAmount"]*-1) if not is_dom else fwd_leg_df["Ccy2Amount"],
                #### Cost- An amount of a specific currency, specifying a value and an associated unit
                cost=lm.CurrencyAndAmount((fwd_leg_df["CcyAmount"]*-1), fwd_leg_df["Ccy"]) if not is_dom else lm.CurrencyAndAmount(
                    fwd_leg_df["Ccy2Amount"], fwd_leg_df["Ccy2"]),
                ###The total cost of the tax-lot in the transaction portfolio's base currency.
                portfolio_cost=fwd_leg_df["CostFC"],
                #### The purchase price of each unit of the instrument held in this tax-lot. This forms part of the unique key required for multiple tax-lots.
                price=fwd_leg_df["Price"],
                #### The purchase date of this tax-lot. This forms part of the unique key required for multiple tax-lots.
                purchase_date=purchase_date,
                #####The settlement date of the tax-lot's opening transaction.
                settlement_date=settlement_date,
            )
        ]
        sub_holding_keys=None
        return lm.AdjustHoldingRequest(
            instrument_identifiers,
            sub_holding_keys,
            properties,
            tax_lots
        )

# setup loader
fx_forward_holding_loaders = FxForwardHoldingLoader(file_path,
                                                    api_factory, 
                                                    Port_scope, 
                                                    'Fx_Port', 
                                                    Holding_scope)

# construct fx fwd requests

effective_at = str(date(2021,3,28))
fx_fwd_requests = [fx_forward_holding_loaders._create_adjust_holdings_requests(fx_fwd) 
                   for i, fx_fwd in fx_fwds.iterrows()][0]
    
# submit fx fwd requests as adjustments (not setting holdings)
fx_forward_holding_loaders.adjust_fx_forward_holdings(effective_at, fx_fwd_requests)

,instrument_identifiers.Instrument/default/Currency,properties.Holding/default/ForwardTransactionId.key,properties.Holding/default/ForwardTransactionId.value.label_value,tax_lots.0.units,tax_lots.0.cost.amount,tax_lots.0.cost.currency,tax_lots.0.portfolio_cost,tax_lots.0.price,tax_lots.0.purchase_date,tax_lots.0.settlement_date
0,GBP,Holding/default/ForwardTransactionId,101,6850,6850,USD,6850,1.37,2021-01-01T00:00:00+00:00,2021-05-01T00:00:00+00:00
1,USD,Holding/default/ForwardTransactionId,101,-5000,-5000,GBP,6850,1.37,2021-01-01T00:00:00+00:00,2021-05-01T00:00:00+00:00


{'response link': 'http://steco.lusid.com/app/insights/logs/0HMRGJDLGN4L6:00000013'}

## 3. Upsert FX Forward position as Transaction

Function to upsert an fx forward position as a transaction to the portfolio created above. This will create
separate cash balances with holding type F which will be booked at maturity.

In [14]:
def upsert_fx(filename):
    swap_df=pd.read_csv(filename)
    T_id= 'Transaction_1'
    request= lm.TransactionRequest(
    transaction_id=T_id,
    type="FwdFxBuy",
    instrument_identifiers={"Instrument/default/Currency": swap_df['Ccy'].item()},
    transaction_date=str(date(2021,4,1)) ,
    settlement_date=str(date(2021,5,1)),
    units=swap_df['CcyAmount'].item(),
    transaction_price=lm.TransactionPrice(1,'Price'),
    total_consideration=lm.CurrencyAndAmount(swap_df['Ccy2Amount'].item(),swap_df['Ccy2'].item()))
    response= transactions_api.upsert_transactions('FX_FW_Transact',
                                                   Port1_Code,
                                                   transaction_request=[request])
    response= response.to_dict()
    return {'response link':response['links'][2]['href']}

In [15]:
upsert_fx(file_path)

{'response link': 'http://steco.lusid.com/app/insights/logs/0HMRGJDLGN4L6:00000014'}

## 4. Valuation

Upsert quotes for fwd rates this will be an input for the valuation we are running at the end

### 4.1 Load FWD Rate

In [16]:
# set up the quotes
effective_date = str(date(2021, 4, 4))
fx_fwd_series_id = lm.QuoteSeriesId(
    provider="Lusid",
    instrument_id="GBP/USD/FxFwdRate/20210501",
    instrument_id_type="LusidInstrumentId",
    quote_type="Price",
    field="mid"
)

fx_fwd_quote = lm.QuoteId(
    quote_series_id=fx_fwd_series_id,
    effective_at=effective_date
)

fx_fwd_quote_request = lm.UpsertQuoteRequest(
    quote_id=fx_fwd_quote,
    metric_value=lm.MetricValue(
        value=1.6, unit="USD"),
    lineage="default"
)

response = quotes_api.upsert_quotes(scope="FxData",
                                     request_body={"1": fx_fwd_quote_request}
                                     )
display(lusid_response_to_data_frame(response))

,response_values
href,None
values.1.quote_id.quote_series_id.provider,Lusid
values.1.quote_id.quote_series_id.price_source,None
values.1.quote_id.quote_series_id.instrument_id,GBP/USD/FxFwdRate/20210501
values.1.quote_id.quote_series_id.instrument_id_type,LusidInstrumentId
values.1.quote_id.quote_series_id.quote_type,Price
values.1.quote_id.quote_series_id.field,mid
values.1.quote_id.effective_at,2021-04-04T00:00:00.0000000+00:00
values.1.metric_value.value,1.6
values.1.metric_value.unit,USD


### 4.2 Quote for spot

In [17]:
# set up the quotes
effective_date = str(date(2021, 4, 4))
fx_fwd_series_id = lm.QuoteSeriesId(
    provider="Lusid",
    instrument_id="GBP/USD",
    instrument_id_type="CurrencyPair",
    quote_type="Price",
    field="mid"
)

fx_fwd_quote = lm.QuoteId(
    quote_series_id=fx_fwd_series_id,
    effective_at=effective_date
)

fx_fwd_quote_request = lm.UpsertQuoteRequest(
    quote_id=fx_fwd_quote,
    metric_value=lm.MetricValue(
        value=1.41, unit="USD"),
    lineage="default"
)

response = quotes_api.upsert_quotes(scope="FxData",
                                     request_body={"1": fx_fwd_quote_request}
                                     )
display(lusid_response_to_data_frame(response))

,response_values
href,None
values.1.quote_id.quote_series_id.provider,Lusid
values.1.quote_id.quote_series_id.price_source,None
values.1.quote_id.quote_series_id.instrument_id,GBP/USD
values.1.quote_id.quote_series_id.instrument_id_type,CurrencyPair
values.1.quote_id.quote_series_id.quote_type,Price
values.1.quote_id.quote_series_id.field,mid
values.1.quote_id.effective_at,2021-04-04T00:00:00.0000000+00:00
values.1.metric_value.value,1.41
values.1.metric_value.unit,USD


## 4.3 Define Recipe for valuation

Define the Recipe that will be used by the Aggregation engine to calculate a valuation for the portfolio. You can learn more about recipes <a id='https://support.lusid.com/knowledgebase/article/KA-01895/en-us'>here</a>.

In [18]:
recipe_scope="Finbourne-Examples"
recipe_code="FxForwards"

rules = [lm.VendorModelRule(supplier='Lusid', model_name='ForwardSpecifiedRateUndiscounted',instrument_type='Future',parameters="{}"),
        lm.VendorModelRule(supplier='Lusid', model_name='ConstantTimeValueOfMoney', instrument_type='CashSettled',parameters="{}")]

config_recipe = lm.ConfigurationRecipe(
        scope=recipe_scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key='Fx.CurrencyPair.*',
                    data_scope='FxData',
                    supplier='Lusid',
                    quote_type='Price',
                    quote_interval='1D.0D',
                    field='mid'),
                lm.MarketDataKeyRule(
                    key='Fx.*.*',
                    data_scope='FxData',
                    supplier='Lusid',
                    quote_type='Price',
                    quote_interval='1D.0D',
                    field='mid'),
                  lm.MarketDataKeyRule(
                    key='Quote.*.*',
                    data_scope='FxData',
                    supplier='Lusid',
                    quote_type='Price',
                    quote_interval='1D.0D',
                    field='mid'), 
            ],
            options=lm.MarketOptions(
                default_supplier='Lusid',
                default_instrument_code_type='Figi',
                default_scope='Lusid',
                attempt_to_infer_missing_fx=True
            ),
        ),
        pricing=lm.PricingContext(
            options=lm.PricingOptions(
                model_selection=lm.ModelSelection(
                    library="Lusid",
                    model="ForwardSpecifiedRateUndiscounted"
                ),
                produce_separate_result_for_linear_otc_legs=True,
            ),
        model_rules=rules),
        description="ifrs test"
    )

response = configuration_recipe_api.upsert_configuration_recipe(
    lm.UpsertRecipeRequest(configuration_recipe=config_recipe)
)
display(lusid_response_to_data_frame(response))

,response_values
href,None
value,2023-06-20 12:26:09.027050+00:00
links.0.relation,RequestLogs
links.0.href,http://steco.lusid.com/app/insights/logs/0HMRG...
links.0.description,A link to the LUSID Insights website showing a...
links.0.method,GET


## 4.4. Run Valuation

Construct a valuation request specifying which metrics we would like included. This request is then passed to the Lusid aggregation API.
The PV of the forward position is shown in forward space (valuation is converted back to GBP using the fwd rate).

In [31]:
valuation_date = str(date(2021, 4, 4))

# Create the valuation request
valuation_request = lm.ValuationRequest(
    recipe_id= lm.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        lm.AggregateSpec("Holding/default/Cost", "Value"),
        lm.AggregateSpec("Holding/Cost/Pfolio", "Value"),
        lm.AggregateSpec("Valuation/PV/Ccy", "Value"),
        lm.AggregateSpec("Valuation/PV/Amount", "Value"),
        lm.AggregateSpec("Valuation/PvInPortfolioCcy/Amount", "Value")
    ],
    portfolio_entity_ids=[lm.PortfolioEntityId(
        scope='FX_FW_Transact', 
        code=Port1_Code)
                         ],
    valuation_schedule=lm.ValuationSchedule(effective_at=valuation_date)
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request)

pd.DataFrame(valuation.data)

,Valuation/PV/Ccy,Valuation/PV/Amount,Valuation/PvInPortfolioCcy/Amount,Holding/default/Cost,Holding/Cost/Pfolio
0,GBP,10000.0000,10000.00,10000.0,10000.0
1,GBP,6850.0000,6850.00,6850.0,6850.0
2,USD,-4406.2500,-3125.00,-5000.0,-6850.0
3,GBP,5000.0000,5000.00,5000.0,5000.0
4,USD,-6036.5625,-4281.25,-6850.0,-5000.0


# Get Holdings

In [24]:
df=lusid_response_to_data_frame(transactions_api.get_holdings(scope='FX_FW_Transact',code='Fx_Port',effective_at=date(2021, 4, 4)))

df=df[['holding_type','units', 'settled_units', 'currency','transaction.settlement_date']]
df

,holding_type,units,settled_units,currency,transaction.settlement_date
0,B,10000.0,10000.0,GBP,NaT
1,F,6850.0,0.0,GBP,2021-05-01 00:00:00+00:00
2,F,-5000.0,0.0,USD,2021-05-01 00:00:00+00:00
3,F,5000.0,0.0,GBP,2021-05-01 00:00:00+00:00
4,F,-6850.0,0.0,USD,2021-05-01 00:00:00+00:00


### Querying for holdings after the settlement date of the forwards

(Notice how LUSID has automatically moved the cash into settled cash buckets)

In [23]:
df = lusid_response_to_data_frame(transactions_api.get_holdings(scope='FX_FW_Transact',code='Fx_Port',effective_at=date(2021, 5, 2)))

df[['holding_type','units', 'settled_units', 'currency']]

,holding_type,units,settled_units,currency
0,B,21850.0,21850.0,GBP
1,B,-11850.0,-11850.0,USD
